![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# **FewShotClassifierModel**

This notebook covers the uses of `FewShotClassifierModel`. This annotator specifically targets few-shot classification tasks, which involves training models to make accurate predictions with limited labeled data.




**📖 Learning Objectives:**

1. Understand how `FewShotClassifierModel` works.

2. Become comfortable using the parameters of the annotator.


**🔗 Helpful Links:**

- Documentation : [FewShotClassifierModel](https://nlp.johnsnowlabs.com/docs/en/licensed_annotators#fewshotclassifier)

- Python Docs : [FewShotClassifierModel](https://nlp.johnsnowlabs.com/licensed/api/python/reference/autosummary/sparknlp_jsl/annotator/classification/few_shot_classifier/index.html#sparknlp_jsl.annotator.classification.few_shot_classifier.FewShotClassifierModel)

- Scala Docs : [FewShotClassifierModel](https://nlp.johnsnowlabs.com/licensed/api/com/johnsnowlabs/nlp/annotators/classification/FewShotClassifierModel.html)

- For extended examples of usage, see [Spark NLP Workshop repository](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/30.3.Text_Classification_with_FewShotClassifier.ipynb#scrollTo=REDlKd7enG5r).


## **📜 Background**

`FewShotClassifierModel` provides a valuable capability for handling scenarios where labeled data is scarce or expensive to obtain. By effectively utilizing limited labeled examples, the few-shot classification approach enables the creation of models that can generalize and classify new instances accurately, even with minimal training data.

`FewShotClassifier` is designed to process sentence embeddings as an input. It generates category annotations, providing labels along with confidence scores that range from 0 to 1. Input annotation types supported by this model include `SENTENCE_EMBEDDINGS`, while the output annotation type is `CATEGORY`.

## **🎬 Colab Setup**

In [ ]:
# Install the johnsnowlabs library to access Spark-OCR and Spark-NLP for Healthcare, Finance, and Legal.
! pip install -q johnsnowlabs

In [ ]:
from google.colab import files
print('Please Upload your John Snow Labs License using the button below')
license_keys = files.upload()

In [ ]:
from johnsnowlabs import nlp, medical

# After uploading your license run this to install all licensed Python Wheels and pre-download Jars the Spark Session JVM
nlp.install()

In [ ]:
from johnsnowlabs import nlp, medical
import pandas as pd

# Automatically load license data and start a session with all jars user has access to
spark = nlp.start()

In [ ]:
from pyspark.sql import DataFrame
import pyspark.sql.functions as F
import pyspark.sql.types as T
import pyspark.sql as SQL
from pyspark import keyword_only

## **🖨️ Input/Output Annotation Types**

- Input: `SENTENCE EMBEDDINGS`

- Output: `CATEGORY`

## **🔎 Parameters**

- `setFeatureScaling`:  Feature scaling method. Possible values are 'zscore', 'minmax' or empty (no scaling)").
- `setMultiClass`: If multiClass is set, the model will return all the labels with corresponding scores. By default, multiClass is false.   



## **💻 Pipeline**

We will use this pretrained model [Few Shot Patient Complaint Classification](https://nlp.johnsnowlabs.com/2023/08/30/few_shot_classifier_patient_complaint_sbiobert_cased_mli_en.html) in the pipeline to understand the few shot classification concept.


This Text Classifier model was trained using healthcare-related text and  google reviews of various healthcare facilities written by patients about the performance of the facility and its personnel.

The dataset has been labeled with two different classes:

`Complaint`: The text includes dissatisfaction or frustration with some aspect of the healthcare provided to the patient. Most often, negative or critical language is used to describe the experience,

`No_Complaint`: The review expresses positive or neutral sentiment about the service. There is no criticism or expressing of dissatisfaction.

In [ ]:
# Annotator that transforms a text column from dataframe into an Annotation ready for NLP
document_assembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

bert_sent = nlp.BertSentenceEmbeddings.pretrained("sbiobert_base_cased_mli", "en", "clinical/models")\
    .setInputCols(["document"])\
    .setOutputCol("sentence_embeddings")

few_shot_classifier = medical.FewShotClassifierModel.pretrained("few_shot_classifier_patient_complaint_sbiobert_cased_mli", "en", "clinical/models")\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("prediction")

clf_Pipeline = nlp.Pipeline(stages=[
    document_assembler,
    bert_sent,
    few_shot_classifier
])

Let us use those sample texts, which may or may not include patients' complaints, to test the performance of the healthcare facility and its personnel.

We will convert the text to Pyspark dataframe and then get predictions for complaints by using `.transform`.

In [ ]:
data = spark.createDataFrame([["""The Medical Center is a large state of the art hospital facility with great doctors, nurses, technicians and receptionists.  Service is top notch, knowledgeable and friendly.  This hospital site has plenty of parking"""],
 ["""My gf dad wasn’t feeling well so we decided to take him to this place cus it’s his insurance and we waited for a while and mind that my girl dad couldn’t breath good while the staff seem not to care and when they got to us they said they we’re gonna a take some blood samples and they made us wait again and to see the staff workers talking to each other and laughing taking there time and not seeming to care about there patience, while we were in the lobby there was another guy who told us they also made him wait while he can hardly breath and they left him there to wait my girl dad is coughing and not doing better and when the lady came in my girl dad didn’t have his shirt because he was hot and the lady came in said put on his shirt on and then left still waiting to get help rn"""],
 ["The doctor seemed rushed during the appointment, and I didn't get a chance to discuss all my concerns. I feel unheard."],
 ["I don't know if it's just me, but the treatment plan doesn't seem to be working, and I'm still in a lot of pain."],
 ["I can't say enough good things about the hospital staff. They were all incredibly kind, compassionate, and went above and beyond to ensure my comfort during my stay. The nurses were always available and responsive to my needs, and the doctors clearly explained everything in a way that was easy to understand."],
 ["The equipment and resources available to the medical staff were truly impressive, and I felt confident that I was receiving the best possible care."],
 ["I'm not sure if it's the medication or something else, but I've been experiencing strange side effects like vivid dreams and dizziness."]
                              ]).toDF("text")

In [ ]:
result = clf_Pipeline.fit(data).transform(data)

In [ ]:
result.select('prediction.result','text').show(truncate = 150)

+--------------+------------------------------------------------------------------------------------------------------------------------------------------------------+
|        result|                                                                                                                                                  text|
+--------------+------------------------------------------------------------------------------------------------------------------------------------------------------+
|[No_Complaint]|The Medical Center is a large state of the art hospital facility with great doctors, nurses, technicians and receptionists.  Service is top notch, ...|
|   [Complaint]|My gf dad wasn’t feeling well so we decided to take him to this place cus it’s his insurance and we waited for a while and mind that my girl dad co...|
|   [Complaint]|                                The doctor seemed rushed during the appointment, and I didn't get a chance to discuss all my concerns. I feel un

### `setFeatureScaling`

This parameter is used to define the Feature Scaling method. Possible values are 'zscore', 'minmax' or empty (no scaling)").



In [ ]:
# zscore

few_shot_classifier = medical.FewShotClassifierModel.pretrained("few_shot_classifier_patient_complaint_sbiobert_cased_mli", "en", "clinical/models")\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("prediction")\
    .setFeatureScaling("zscore")

clf_Pipeline = nlp.Pipeline(stages=[
    document_assembler,
    bert_sent,
    few_shot_classifier
])

result = clf_Pipeline.fit(data).transform(data)

In [ ]:
result.select('prediction.result','text').show(truncate = 150)

+--------------+------------------------------------------------------------------------------------------------------------------------------------------------------+
|        result|                                                                                                                                                  text|
+--------------+------------------------------------------------------------------------------------------------------------------------------------------------------+
|[No_Complaint]|The Medical Center is a large state of the art hospital facility with great doctors, nurses, technicians and receptionists.  Service is top notch, ...|
|   [Complaint]|My gf dad wasn’t feeling well so we decided to take him to this place cus it’s his insurance and we waited for a while and mind that my girl dad co...|
|   [Complaint]|                                The doctor seemed rushed during the appointment, and I didn't get a chance to discuss all my concerns. I feel un

In [ ]:
# minmax

few_shot_classifier = medical.FewShotClassifierModel.pretrained("few_shot_classifier_patient_complaint_sbiobert_cased_mli", "en", "clinical/models")\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("prediction")\
    .setFeatureScaling("minmax")

clf_Pipeline = nlp.Pipeline(stages=[
    document_assembler,
    bert_sent,
    few_shot_classifier
])

result = clf_Pipeline.fit(data).transform(data)

In [ ]:
result.select('prediction.result','text').show(truncate = 150)

+--------------+------------------------------------------------------------------------------------------------------------------------------------------------------+
|        result|                                                                                                                                                  text|
+--------------+------------------------------------------------------------------------------------------------------------------------------------------------------+
|[No_Complaint]|The Medical Center is a large state of the art hospital facility with great doctors, nurses, technicians and receptionists.  Service is top notch, ...|
|   [Complaint]|My gf dad wasn’t feeling well so we decided to take him to this place cus it’s his insurance and we waited for a while and mind that my girl dad co...|
|   [Complaint]|                                The doctor seemed rushed during the appointment, and I didn't get a chance to discuss all my concerns. I feel un